# Tests Data & Graphs

## 1 - Introduction

In this file, we're going to import test data from CSV files and generate statistical data, plot interactive graphs and get some insight about the project's scope.

## 2 - Libraries, Packages and Options

We're using Plotly to generate the graphs, Pandas for handling the CSV data files and Numpy for math and statistics operations.

For dynamic and interactive plots, we're using ipywidgets.

We're also setting Plotly as the default plot backend on Pandas

In [1]:
import plotly.express as px  # Plotly express graphs
import plotly.graph_objects as go  # Plotly advanced graphs
import pandas as pd
import numpy as np

from ipywidgets import interact, widgets  # Interactive widgets for Jupyter Notebooks

pd.options.plotting.backend = "plotly"

## 3 - Test Data

Here we can explore the data files generated by CETR UMT tribometer. In this example, we open a polyurethane + 1% graphene nanoplatelets sample test and plot the COF vs. Time.

Filenames are organized by percent of GNPs in the composite (0pct = 0%; 4pct = 4%). In the graph, LV is the position sensor on reciprocating test, in milimeters. For all tests, normal load was 5.17 N and reciprocating frequency was 3 Hz.

Data from the 18 tests was summarized into ``data.csv`` file, which we will use in the following sections.

In [108]:
filenames = [
    "src/Adicionais_0pct_001.csv",
    "src/Adicionais_0pct_002.csv",
    "src/Adicionais_0pct_003.csv",
    "src/Adicionais_0pct_004.csv",
    "src/Adicionais_0pct_005.csv",
    "src/Adicionais_1pct_001.csv",
    "src/Adicionais_1pct_002.csv",
    "src/Adicionais_1pct_003.csv",
    "src/Adicionais_1pct_004.csv",
    "src/Adicionais_2pct_001.csv",
    "src/Adicionais_2pct_002.csv",
    "src/Adicionais_2pct_003.csv",
    "src/Adicionais_2pct_004.csv",
    "src/Adicionais_4pct_001.csv",
    "src/Adicionais_4pct_002.csv",
    "src/Adicionais_4pct_003.csv",
    "src/Adicionais_4pct_004.csv",
    "src/Adicionais_4pct_005.csv",
]

lv_not_working = [
    "src/Adicionais_0pct_001.csv",
    "src/Adicionais_0pct_002.csv",
    "src/Adicionais_0pct_003.csv",
    "src/Adicionais_0pct_004.csv",
    "src/Adicionais_0pct_005.csv",
    "src/Adicionais_4pct_001.csv",
    "src/Adicionais_4pct_002.csv",
    "src/Adicionais_4pct_003.csv",
    "src/Adicionais_4pct_004.csv",
    "src/Adicionais_4pct_005.csv",
]

def plot_3d_COF(filename):
    df = pd.read_csv(filename,
                     header=17,
                     skiprows = [21],
                     dtype = np.float64)
    if filename in lv_not_working:  # LV sensor was not working properly during some earlier tests, this mitigates it
        zero = df.tail(n = df.shape[0]//2).nsmallest(1,'COF').iat[0,0]
        instant_position = 4.66/2 * np.cos(2.*np.pi*3*(df['T']-zero))
        df["LV"] = instant_position
    graph_df = df.sample(frac=0.1)
    fig = px.scatter_3d(
        graph_df,
        x = "T",
        y = "LV",
        z = "COF",
        color = "COF",
        opacity = 0.20,
        labels = {"COF": "COF (-)", "LV": "Position (mm)", "T": "t (s)"},
        template = "presentation"
    ).update_traces(marker_size=2)
    fig.update_scenes(zaxis_range = [0,0.5], camera_eye=dict(x=1.5,y=1.5,z=1.5)).update_layout(width=900, height=900)
    return fig

interact(plot_3d_COF, filename = filenames)

interactive(children=(Dropdown(description='filename', options=('src/Adicionais_0pct_001.csv', 'src/Adicionais…

<function __main__.plot_3d_COF(filename)>

## 4 - Dataframe and Graphs for Average COF

The function ``pandas.read_csv`` opens the tribological test data summary, which contains mean and standard deviation values for COF for all tests.

Note that as we increased GNPs from 0 to 1%, COF was decreased to the same level observed with 2% and 4% GNPs.

In [109]:
filename = 'src/data.csv'
df = pd.read_csv(
    filename,
    dtype = {
        'Força Normal (N)': np.float64,
        'Frequência (Hz)': np.float64
    },
    parse_dates = ['Início']
)
df_stats = df.groupby("Material").describe()["COF Médio (-)"]
df_stats = df_stats.reset_index()

In [110]:
px.bar(df_stats, x = "Material", y = "mean", error_y = "std", labels={"mean": "Mean COF (-)"}, template = "presentation").update_layout(width = 900, height = 900)

In [96]:
df_stats

,Material,count,mean,std,min,25%,50%,75%,max
0,PU,5.0,0.315201,0.034098,0.280333,0.288570,0.308932,0.335115,0.363054
1,PU+1%GNP,4.0,0.195337,0.017262,0.172521,0.188039,0.197822,0.205119,0.213183
2,PU+2%GNP,4.0,0.220201,0.026841,0.181679,0.212005,0.229639,0.237835,0.239846
3,PU+4%GNP,5.0,0.187007,0.005085,0.180929,0.184111,0.187355,0.188099,0.194543
